In [ ]:
import pandas as pd
import numpy as np
import scipy
from scipy.sparse import load_npz
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U pyarrow
!python -m pip install featuretools

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (1.0.1)
     |████████████████████████████████| 296kB 3.4MB/s 
     |████████████████████████████████| 665kB 10.2MB/s 
     |████████████████████████████████| 92kB 7.2MB/s 
     |████████████████████████████████| 102kB 8.0MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=3c397b83e23884d41ccfeac230cd930e616ae79aba68570bf799d14dde603a3a
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=7a6f3ee448d77d9f98a42cc5d94463259c3dcd38110cbbb2ee901253c62f32af
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built contextvars locket
ERROR: distributed 2.30.0 has requirement cloudpickle>=1.5.0, but you'll have cloudpickle 1.3.0 which is incompatible.
  Found exi

In [ ]:
trans = pd.read_parquet('/content/drive/My Drive/sber/Копия all_data.parquet')
trans = trans.drop(columns=['line_item_id','price','discount','master_category_id','parent_category_id'])
trans.head()

,user_id,order_id,quantity,product_name,product_id,brand_name
0,525,5354800,4,Лазанья Золотой Петушок мясной слоеный пирог з...,37548,Золотой Петушок
1,525,5354800,2,Хлеб Хлебный Дом Геркулес зерновой с отрубями ...,5636,Хлебный Дом
2,525,5354800,2,Тортилья Delicados оригинальная пшеничная,22107,Delicados
3,525,5354800,6,Вода питьевая минеральная BonAqua газированная...,2530,BonAqua
4,525,5354800,6,"Газированный напиток Mirinda Апельсин 1,5 л",3818486,Mirinda


In [ ]:
orders = trans[['user_id', 'order_id']].dropna().drop_duplicates()
orders = orders.groupby(['user_id'])['order_id'].count().reset_index()
orders.columns = ['user_id', 'user_order_num']
trans = trans.merge(orders, how='left', on=['user_id'])

In [ ]:
trans_ = trans.groupby(['user_id', 'product_id'])['quantity'].sum().reset_index()

In [ ]:
trans_ = trans_.merge(orders, how='left', on=['user_id'])

In [ ]:
trans_

,user_id,product_id,quantity,user_order_num
0,51,0,9,17
1,51,159,1,17
2,51,397,2,17
3,51,407,16,17
4,51,456,20,17
...,...,...,...,...
46194231,3142966,6685882,1,1
46194232,3142966,6742924,1,1
46194233,3142966,6782782,1,1
46194234,3142966,6803534,1,1


In [ ]:
items_in_trans = trans.groupby(['user_id', 'product_id'])['quantity'].sum().reset_index()
items_in_trans.columns = ['user_id', 'product_id', 'user_products_num']
items_in_trans

,user_id,product_id,user_products_num
0,51,0,9
1,51,159,1
2,51,397,2
3,51,407,16
4,51,456,20
...,...,...,...
46194231,3142966,6685882,1
46194232,3142966,6742924,1
46194233,3142966,6782782,1
46194234,3142966,6803534,1


In [ ]:
items_in_trans = items_in_trans.merge(trans_, how='left', on=['user_id', 'product_id'])

In [ ]:
items_in_trans

,user_id,product_id,user_products_num,quantity,user_order_num
0,51,0,9,9,17
1,51,159,1,1,17
2,51,397,2,2,17
3,51,407,16,16,17
4,51,456,20,20,17
...,...,...,...,...,...
46194231,3142966,6685882,1,1,1
46194232,3142966,6742924,1,1,1
46194233,3142966,6782782,1,1,1
46194234,3142966,6803534,1,1,1


In [ ]:
items_in_trans['mean_items_in_trans'] = items_in_trans.user_products_num / items_in_trans.user_order_num
items_in_trans['mean_items_in_trans_log_denom'] = items_in_trans.user_products_num / (1 + np.log(items_in_trans.user_order_num))
items_in_trans['mean_items_in_trans_log'] = np.log(items_in_trans.user_products_num /items_in_trans.user_order_num + 0.001)


In [ ]:
items_in_trans = items_in_trans.drop(columns=['quantity','user_products_num'])
items_in_trans

,user_id,product_id,user_order_num,mean_items_in_trans,mean_items_in_trans_log_denom,mean_items_in_trans_log
0,51,0,17,0.529412,2.347900,-0.634102
1,51,159,17,0.058824,0.260878,-2.816356
2,51,397,17,0.117647,0.521755,-2.131602
3,51,407,17,0.941176,4.174044,-0.059563
4,51,456,17,1.176471,5.217555,0.163369
...,...,...,...,...,...,...
46194231,3142966,6685882,1,1.000000,1.000000,0.001000
46194232,3142966,6742924,1,1.000000,1.000000,0.001000
46194233,3142966,6782782,1,1.000000,1.000000,0.001000
46194234,3142966,6803534,1,1.000000,1.000000,0.001000


In [ ]:
items_in_trans.to_csv('/content/drive/My Drive/sber/trans_features.csv')